<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_0325_test_MutiLNEuler_5layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Euler 實驗結果測試
<a href="https://drive.google.com/file/d/1RzHAFqRE0sV5uXkwbzjXXo0uOHL8SFRo/view?usp=sharing">請見paper</a><br>
把每一層都做LN<br>
要對 G4 做正規化

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [ ]:
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense, Flatten, LayerNormalization, Input,Activation, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from random import choice, sample
import math


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    sqr_num = int(num ** 0.5)
    # for i in range(2, sqr_num+ 1):
    for i in range(2, sqr_num+ 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

def add_prime(primeTable, limit, addSize=5):
    halfList = [num for num in primeTable if num <= int(limit//2)]
    if len(halfList) < 5:
      halfList += [0] * (5 - len(halfList))
    return sample(halfList, addSize)

def add_mod(primeTable25, num):
  return [num%index for index in primeTable25]

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def compute_coprimes(n): ## 獲得質因數 zero padding 到40
    coprimes = []
    lenth = 50
    for num in range(1,n):
        if gcd(num , n) == 1:
            coprimes.append(num)
        if len(coprimes) >= lenth:
            return coprimes
    coprimes = coprimes + [0] * (lenth - len(coprimes))
    return coprimes


In [ ]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/Drive/MyDrive/實驗/goldbach/csv/traindata_min_size_G4_5000.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


In [ ]:
print (len(trainDataNumbers))

5000


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [ ]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [ ]:
# collect data
# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []

for num in shuffledNumbers:

    # Euler's Totient Function Partitions fist 50
    feature = []
    feature = compute_coprimes(num)
    mergedTrainNumber.append(feature)

for num in shuffledLabel:
    mergedTrainLabel.append(num)

    print("merge number ", num, " done.")

#print (mergedTrainNumber)

In [ ]:
mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)
print ("X Train [0] value : \n",X_train[0])
print ("X Train [0] label : ",y_train[0])
print ("=======")
print ("X shpae : " , X_train[0].shape)

X Train [0] value : 
 [  1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35
  37  39  41  43  45  47  49  51  53  55  57  61  63  65  67  69  71  73
  75  77  79  81  83  85  87  89  91  93  95  97  99 101]
X Train [0] label :  2622.509707
X shpae :  (50,)


In [ ]:
# start training
# 清空所有
K.clear_session()

################################################################

def LN_Layer(x,u1,u2):
  # two group
  group1 = x
  group2 = Activation('relu')(x)
  group2 = Lambda(lambda x: K.log(x + 1))(group2)  # 使用 natural logarithm

  # G1 -> next
  group1 = Dense(units=u1, activation='relu')(group1)

  # G2 -> exp -> next
  group2 = Dense(units=u2, activation='linear')(group2)
  group2 = Activation('exponential')(group2)

  # concate
  merged = concatenate([group1, group2], axis=-1)

  return merged


# require gpu
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    inputLayer = Input(shape=X_train[0].shape)

    # LN_input
    LN_1 = LN_Layer(inputLayer,50,200)

    # LN_2
    LN_2 = LN_Layer(LN_1,120,80)

    # LN_3
    LN_3 = LN_Layer(LN_2,120,80)

    # LN_4
    LN_4 = LN_Layer(LN_3,60,140)

    # LN_5
    LN_5 = LN_Layer(LN_4,60,140)


    #Output
    outputLayer = Dense(units=1, activation='linear')(LN_5)
    model = Model(inputs=inputLayer , outputs=outputLayer)


    print(model.summary())

    custom_adam_optimizer = Adam(learning_rate=0.001)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer,
                  metrics=['accuracy'])    # train version 2

    # train model
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=1000,batch_size=2048, validation_data=(X_val, y_val))
    print("--------------------------------------------------------")

    # save model
    model.save('./model/goldbach_model_0325_.h5')
print("model saved.")


# predict

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")


# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

---loading primes table---
[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [ ]:
## load data
from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_0325_mltiEuler_4layer_2.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set_G4_7M.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values
testDataG4 = testData['G4'].values
testDataPartition = testData['Partition'].values

print("---loading test data done.---")


---loading test data done.---


In [ ]:
testDataNumbers = [100000, 10000, 1000, 1579746]
testDataPartition = [810, 127, 28, 14932]

In [ ]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []

for num in testDataNumbers:
    #Euler's Totient Function Partitions fist 50
    feature = []
    feature = compute_coprimes(num)
    mergedTestNumber.append(feature)

for num in testDataPartition:
    mergedTestLabel.append(num)

In [ ]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [ ]:
evaluation = model.evaluate(mergedArray, mergedLabel)
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)



2/2 [==============================] - 1s 45ms/step - loss: 1433156480.0000 - accuracy: 0.0000e+00
Loss: 1433156480.0
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 9ms/step


In [ ]:
"""
順便計算了G4
本模型的訓練目的是建立原始分區與 G4 之間的關係，
因此標籤資料需要除以 G4。下方的測試資料中，數值 7984914 的正確答案是 58853，而模型預測值為 1.721389。

實際上，正確答案應該是 1.721389 乘以 G4，這樣才能得到模型的最終正確答案。
"""
# cal G4
def G4(n) -> int:
  return int((n / np.log((n/2))**2)+0.5)

testDataG4 = [G4(i) for i in testDataNumbers]


print (f"測試五十筆資料")
roundPredictions = np.round(predictions)
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案為:{testDataPartition[index]}\t模型預估為{predictions[index]}, 四捨五入到{roundPredictions[index]}")

測試五十筆資料
第0筆資料為數字:	7984914	|正確答案為:58853	模型預估為[1.721389], 四捨五入到[2.]
第1筆資料為數字:	7050792	|正確答案為:51667	模型預估為[1.6746786], 四捨五入到[2.]
第2筆資料為數字:	7852768	|正確答案為:31367	模型預估為[0.9302076], 四捨五入到[1.]
第3筆資料為數字:	7807448	|正確答案為:26161	模型預估為[0.7781038], 四捨五入到[1.]
第4筆資料為數字:	7695230	|正確答案為:31001	模型預估為[0.9379368], 四捨五入到[1.]
第5筆資料為數字:	7109000	|正確答案為:28859	模型預估為[0.9379368], 四捨五入到[1.]
第6筆資料為數字:	7268430	|正確答案為:64326	模型預估為[2.0327833], 四捨五入到[2.]
第7筆資料為數字:	7147384	|正確答案為:22306	模型預估為[0.71668863], 四捨五入到[1.]
第8筆資料為數字:	7555772	|正確答案為:27371	模型預估為[0.8386079], 四捨五入到[1.]
第9筆資料為數字:	7066874	|正確答案為:21654	模型預估為[0.7017362], 四捨五入到[1.]
第10筆資料為數字:	7632520	|正確答案為:36879	模型預估為[1.1266344], 四捨五入到[1.]
第11筆資料為數字:	7039812	|正確答案為:46596	模型預估為[1.5313977], 四捨五入到[2.]
第12筆資料為數字:	7670862	|正確答案為:46176	模型預估為[1.4010736], 四捨五入到[1.]
第13筆資料為數字:	7402232	|正確答案為:22333	模型預估為[0.7017362], 四捨五入到[1.]
第14筆資料為數字:	7432508	|正確答案為:22866	模型預估為[0.7137461], 四捨五入到[1.]
第15筆資料為數字:	7230862	|正確答案為:22140	模型預估為[0.7017362], 四捨五入到[1.]
第16筆資料為數字:	7305346	|正確答案為:22305	模型預估為[0.70

# +N 才要執行正規

In [ ]:
"""
+n 調整
+n 代表對原本的partition進行一些修改。
在這裡，+n 是將partition針對 G4 進行正規化，也就是將標籤 (label) 除以 G4。因此，我們需要在這一段將標籤正規化回來。

這就是為什麼在計算中包含了乘以 testDataG4 的部分。
"""
print (f"將50筆測資正規化回G4")
normalizedTest = []
normalizedPred = []
for index in range(len(testDataLabel)):
  normalizedTest.append(testDataLabel[index]*testDataG4[index]) # normalize ground truth
  normalizedPred.append(predictions[index]*testDataG4[index])  # normalize predition



將50筆測資正規化回G4


In [ ]:
"""
正規化後的測試
"""
print (f"將50筆測資正規化回G4")
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")

將50筆測資正規化回G4
第0筆資料為數字:	7984914	|正確答案 = 58853	模型預估 = 59492
第1筆資料為數字:	7050792	|正確答案 = 51667	模型預估 = 51955
第2筆資料為數字:	7852768	|正確答案 = 31367	模型預估 = 31686
第3筆資料為數字:	7807448	|正確答案 = 26161	模型預估 = 26372
第4筆資料為數字:	7695230	|正確答案 = 31001	模型預估 = 31392
第5筆資料為數字:	7109000	|正確答案 = 28859	模型預估 = 29306
第6筆資料為數字:	7268430	|正確答案 = 64326	模型預估 = 64750
第7筆資料為數字:	7147384	|正確答案 = 22306	模型預估 = 22498
第8筆資料為數字:	7555772	|正確答案 = 27371	模型預估 = 27626
第9筆資料為數字:	7066874	|正確答案 = 21654	模型預估 = 21813
第10筆資料為數字:	7632520	|正確答案 = 36879	模型預估 = 37441
第11筆資料為數字:	7039812	|正確答案 = 46596	模型預估 = 47445
第12筆資料為數字:	7670862	|正確答案 = 46176	模型預估 = 46765
第13筆資料為數字:	7402232	|正確答案 = 22333	模型預估 = 22708
第14筆資料為數字:	7432508	|正確答案 = 22866	模型預估 = 23179
第15筆資料為數字:	7230862	|正確答案 = 22140	模型預估 = 22252
第16筆資料為數字:	7305346	|正確答案 = 22305	模型預估 = 22450
第17筆資料為數字:	7349124	|正確答案 = 47254	模型預估 = 47959
第18筆資料為數字:	7134356	|正確答案 = 22965	模型預估 = 23475
第19筆資料為數字:	7990454	|正確答案 = 23942	模型預估 = 24267
第20筆資料為數字:	7024954	|正確答案 = 22002	模型預估 = 21941
第21筆資料為數字:	7715734	|正確答案 = 2551

<ipython-input-14-7ef344336dce>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")


In [ ]:
"""
顯示尚未正規化前的資料
"""
print ("\t正確答案  "  , "\t/" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t/  \t{roundPredictions[index]} = {testDataLabel[index]/roundPredictions[index]} ")
print ("###########################################################################")

print ("\t正確答案  "  , "\t-" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t-  \t{roundPredictions[index]} = {testDataLabel[index]-roundPredictions[index]} ")

	正確答案   	/   	模型預測
0	1.702867347  		/  	[2.] = [0.8514337] 
1	1.665401625  		/  	[2.] = [0.8327008] 
2	0.920828024  		/  	[1.] = [0.92082804] 
3	0.771866828  		/  	[1.] = [0.7718668] 
4	0.926237389  		/  	[1.] = [0.9262374] 
5	0.923612865  		/  	[1.] = [0.9236129] 
6	2.019479073  		/  	[2.] = [1.0097395] 
7	0.71056174  		/  	[1.] = [0.71056175] 
8	0.830867449  		/  	[1.] = [0.83086747] 
9	0.696603547  		/  	[1.] = [0.69660354] 
10	1.109712659  		/  	[1.] = [1.1097126] 
11	1.503977776  		/  	[2.] = [0.7519889] 
12	1.383434641  		/  	[1.] = [1.3834347] 
13	0.690122259  		/  	[1.] = [0.69012225] 
14	0.704094356  		/  	[1.] = [0.70409435] 
15	0.698204988  		/  	[1.] = [0.698205] 
16	0.697181913  		/  	[1.] = [0.69718194] 
17	1.469369561  		/  	[1.] = [1.4693695] 
18	0.732712976  		/  	[1.] = [0.732713] 
19	0.692326664  		/  	[1.] = [0.69232666] 
20	0.711460457  		/  	[1.] = [0.7114605] 
21	0.760450346  		/  	[1.] = [0.76045036] 
22	1.665650216  		/  	[2.] = [0.8328251] 
23	2.019310047  		/

In [ ]:
"""
顯示正確答案與預測答案的差距量
"""
D = []
for index in range (len(testDataPartition)):
  D.append(testDataPartition[index] - int(normalizedPred[index]))


print ("\t正確答案  "  , "\t-" ,"  \t模型預測\t\t差距量")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataPartition[index]}\t\t\t{int(normalizedPred[index])}\t=\t{abs(D[index])}")

	正確答案   	-   	模型預測		差距量
0	58853			59492	=	639
1	51667			51955	=	288
2	31367			31686	=	319
3	26161			26372	=	211
4	31001			31392	=	391
5	28859			29306	=	447
6	64326			64750	=	424
7	22306			22498	=	192
8	27371			27626	=	255
9	21654			21813	=	159
10	36879			37441	=	562
11	46596			47445	=	849
12	46176			46765	=	589
13	22333			22708	=	375
14	22866			23179	=	313
15	22140			22252	=	112
16	22305			22450	=	145
17	47254			47959	=	705
18	22965			23475	=	510
19	23942			24267	=	325
20	22002			21941	=	61
21	25511			25808	=	297
22	53823			54115	=	292
23	65325			65866	=	541
24	23978			24215	=	237
25	45171			45715	=	544
26	22227			22411	=	184
27	46283			46616	=	333
28	23166			23260	=	94
29	22558			22854	=	296
30	56207			56614	=	407
31	28538			29093	=	555
32	30025			30708	=	683
33	21786			22079	=	293
34	33860			34467	=	607
35	46031			46267	=	236
36	32376			32361	=	15
37	25863			26294	=	431
38	43561			44140	=	579
39	21553			21734	=	181
40	21393			21634	=	241
41	44362			44719	=	357
42	51956			52256	=	300


<ipython-input-17-1021a7c2349b>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  D.append(testDataPartition[index] - int(normalizedPred[index]))
<ipython-input-17-1021a7c2349b>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print (f"{index}\t{testDataPartition[index]}\t\t\t{int(normalizedPred[index])}\t=\t{abs(D[index])}")


## 把所有資料zip

In [ ]:
"""
顯示正確答案與預測答案的差距量
"""
gap = [abs(int(partition - pred)) for partition, pred in zip(testDataPartition, normalizedPred)]


normalizedPred_int = [int(arr) for arr in normalizedPred]
combined_data = list(zip(testDataNumbers, testDataPartition, normalizedPred_int, gap))
sort_data = sorted(combined_data, key = lambda x : x[0])


print (f"test on {testDataPath[39::]}")
print (f"測試數字/正確答案/預測答案/差距量\n")
for i in sort_data:
  print (i)

test on test_set_G4_7M.csv
測試數字/正確答案/預測答案/差距量

(7000034, 21393, 21634, 241)
(7024954, 22002, 21941, 60)
(7037344, 21553, 21734, 181)
(7039812, 46596, 47445, 849)
(7049660, 28538, 29093, 555)
(7050792, 51667, 51955, 288)
(7066874, 21654, 21813, 159)
(7097916, 51956, 52256, 300)
(7109000, 28859, 29306, 447)
(7121414, 22558, 22854, 296)
(7134356, 22965, 23475, 510)
(7147384, 22306, 22498, 192)
(7166374, 21786, 22079, 293)
(7177008, 43561, 44140, 579)
(7229336, 22227, 22411, 184)
(7230862, 22140, 22252, 112)
(7236930, 59226, 59533, 307)
(7243074, 44362, 44719, 357)
(7259704, 23166, 23260, 94)
(7268224, 22013, 22351, 338)
(7268430, 64326, 64750, 424)
(7305346, 22305, 22450, 145)
(7349124, 47254, 47959, 705)
(7389732, 53823, 54115, 292)
(7399392, 65325, 65866, 541)
(7402232, 22333, 22708, 375)
(7432508, 22866, 23179, 313)
(7472958, 45171, 45715, 544)
(7502200, 30025, 30708, 683)
(7555772, 27371, 27626, 255)
(7576878, 46031, 46267, 236)
(7606482, 50630, 51152, 522)
(7632520, 36879, 37441, 562

<ipython-input-20-048f8f5163d8>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gap = [abs(int(partition - pred)) for partition, pred in zip(testDataPartition, normalizedPred)]
<ipython-input-20-048f8f5163d8>:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  normalizedPred_int = [int(arr) for arr in normalizedPred]


## RMSE and error rate

In [ ]:
"""
計算三參數
MSE RMSE 跟 error rate
RMSE = sqrt(MSE)
error = RMSE / mean of partition
"""
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse



mean = testData['Partition'].mean()

#LN MSE
MSE = int(calculate_mse(normalizedPred, testDataPartition))

#MLP MSE
# MSE = int(calculate_mse(predictions, testDataPartition))

RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE} / ({MSE/1000000})\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

MSE : 163631 / (0.163631)
RMSE : 404.5132877916373
Error Rate : 0.01148917831049691  ->  1.14892%


<ipython-input-24-2d4cd73154ce>:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  MSE = int(calculate_mse(normalizedPred, testDataPartition))
